# Uncertainty Losses Iteration #2

In [1]:
import os
import sys

sys.path.append(os.path.abspath('..'))

In [2]:
seed = 2018

In [3]:
import random

import numpy as np
from copy import copy
import skml
from skml.problem_transformation import ProbabilisticClassifierChain
from skml.datasets import sample_down_label_space
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.base import clone
from sklearn.externals import joblib
import arff
import os
import time

from lib.experimental_framework import load_data
random.seed(2018)

In [4]:
DATA_SETS = ['scene', 'emotions', 'yeast-10', 'mediamill-10', 'enron-10', 'medical-10', 'slashdot-10',
            'ohsumed-10', 'tmc2007-500-10', 'imdb-10']

In [ ]:
for ds in DATA_SETS:
    tic = time.time()
    data = load_data(ds)
    toc = time.time()
    print("{}: {:.4f} s".format(ds, toc - tic))

In [5]:
def uncertain_hamming_loss(y, y_pred, omega=1.0):
    N, L = y_pred.shape
    cumsum = 0
    
    # place '?' if not already done
    np.place(y_pred, 
             mask=np.logical_and(y_pred > 1/3, y_pred < 2/3),
             vals=np.nan)
    u = np.isnan(y_pred).sum()
    hl = ((y_pred>= .5) != y.astype(float)).sum()
    print("hl", hl / (N * L))
    return (hl + (u * omega)) / (N * L)

In [6]:
def evaluate(y_test, y_pred, y_pred_pp):
    print("----------")

    print("hamming loss: ")
    print(hamming_loss(y_test, y_pred))

    print("accuracy:")
    print(accuracy_score(y_test, y_pred))

    print("f1 score:")
    print("micro")
    print(f1_score(y_test, y_pred, average='micro'))
    print("macro")
    print(f1_score(y_test, y_pred, average='macro'))

    print("precision:")
    print("micro")
    print(precision_score(y_test, y_pred, average='micro'))
    print("macro")
    print(precision_score(y_test, y_pred, average='macro'))

    print("recall:")
    print("micro")
    print(recall_score(y_test, y_pred, average='micro'))
    print("macro")
    print(recall_score(y_test, y_pred, average='macro'))
    
    print("#--")
    print("-> hamming loss:")
    print(hamming_loss(y_test, (y_pred_pp >= .5)))
    print("-> uncertain hamming loss:")
    print(uncertain_hamming_loss(y_test, y_pred_pp))
    print("---")

In [ ]:
splits = {}

for ds in DATA_SETS:
    tic = time.time()
    data = load_data(ds)

    try:
        # FIXME: ohsumed cannot be unpacked like the others
        if len(data) == 1:
            data = data[0]
            
        assert len(data) == 2

        if data[1].shape[1] > 10:
            X, y = data
            y = sample_down_label_space(y, 10)
        else:
            X, y = data
        
        X = X.toarray()
        y = y.toarray()

        if X.shape[0] >= 10000:
            # 66% split
            X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                                test_size=0.33,
                                                                random_state=seed)
            clf = ProbabilisticClassifierChain(LogisticRegression())
            pcc = clone(clf)

            pcc.fit(X_train, y_train)
            y_pred = pcc.predict(X_test)
            y_pred_pp = pcc.predict_proba(X_test)
            y_pred_pp = y_pred_pp.reshape(y_pred_pp.shape[0], y_pred_pp.shape[2])
            print(ds)
            evaluate(y_test, y_pred, y_pred_pp)

        else:
            kf = KFold(random_state=seed)
            results = []

            for train_index, test_index in kf.split(X):
                clf = ProbabilisticClassifierChain(LogisticRegression())
                X_train = X[train_index]
                X_test = X[test_index]
                y_train = y[train_index]
                y_test = y[test_index]

                # FIXME:
                # idxs.append((train_index, test_index))
                pcc = clone(clf)

                pcc.fit(X_train, y_train)
                y_pred = pcc.predict(X_test)
                y_pred_pp = pcc.predict_proba(X_test)
                y_pred_pp = y_pred_pp.reshape(y_pred_pp.shape[0], y_pred_pp.shape[2])

                # FIXME:
                # results.append((y_pred, y_pred_pp))
                print(ds)
                evaluate(y_test, y_pred, y_pred_pp)
    except Exception as e:
        print(e)

    toc = time.time()
    print("{}: {:.4f} s".format(ds, toc - tic))

scene
----------
hamming loss: 
0.10107928601079286
accuracy:
0.6562889165628891
f1 score:
micro
0.7150380339379755
macro
0.6276118090224297
precision:
micro
0.7256532066508313
macro
0.6672593950134255
recall:
micro
0.7047289504036909
macro
0.6786993781672516
#--
-> hamming loss:
0.8233706932337069
-> uncertain hamming loss:
hl 0.8208800332088003
0.8812785388127854
---


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in greater_equal
  # Remove the CWD from sys.path while we load stuff.


scene
----------
hamming loss: 
0.12489609310058188
accuracy:
0.600997506234414
f1 score:
micro
0.6386049308478653
macro
0.5017062723228454
precision:
micro
0.6436363636363637
macro
0.5314905358176077
recall:
micro
0.6336515513126492
macro
0.5592795813338908
#--
-> hamming loss:
0.8243973399833749
-> uncertain hamming loss:
hl 0.806525353283458
0.8767664172901081
---


/usr/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in greater_equal
  # Remove the CWD from sys.path while we load stuff.


scene
----------
hamming loss: 
0.11845386533665836
accuracy:
0.600997506234414
f1 score:
micro
0.6650998824911869
macro
0.5136778823344651
precision:
micro
0.6885644768856448
macro
0.5513059909629657
recall:
micro
0.6431818181818182
macro
0.5628043519372148
#--
-> hamming loss:
0.8123441396508728
-> uncertain hamming loss:
hl 0.801330008312552
0.8740648379052369
---
scene: 102.5374 s


...

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in greater_equal
  # Remove the CWD from sys.path while we load stuff.


ohsumed-10
----------
hamming loss: 
0.09697628888405482
accuracy:
0.3630628670872308
f1 score:
micro
0.5613931523022433
macro
0.5472196469440009
precision:
micro
0.6295233892321271
macro
0.6166979397061072
recall:
micro
0.5065696022727273
macro
0.49373552063452186
#--
-> hamming loss:
0.8762236241026756
-> uncertain hamming loss:
hl 0.8610180552534261
0.9226234500761367
---
ohsumed-10: 5141.6709 s


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in greater_equal
  # Remove the CWD from sys.path while we load stuff.


tmc2007-500-10
----------
hamming loss: 
0.10256437427148458
accuracy:
0.3837024478118046
f1 score:
micro
0.7220036189217911
macro
0.6614089365203334
precision:
micro
0.7467324144486692
macro
0.7273738899312006
recall:
micro
0.6988601612454823
macro
0.6177292371971519
#--
-> hamming loss:
0.8100349687400658
-> uncertain hamming loss:
hl 0.8144855356575182
0.9132563314612695
---
tmc2007-500-10: 10393.4316 s


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in greater_equal
  # Remove the CWD from sys.path while we load stuff.


In [41]:
clf = ProbabilisticClassifierChain(LogisticRegression())

In [71]:
X_train, y_train = np.array([[2,4], [1,3], [6,8], [5,7], [1,1]]), np.array([[1, 1, 1], [0, 1, 0], [1, 1, 1], [0, 1, 0], [1,0,1]])

In [72]:
clf.fit(X_train, y_train)

In [88]:
yp = clf.predict(X_train[1,:])

ValueError: X has 1 features per sample; expecting 2

In [75]:
ypp = clf.predict_proba(X_train)

ValueError: X has 3 features per sample; expecting 2

In [101]:
lr = LogisticRegression()
lr.fit(np.hstack((X_train, y_train[:, :2])), y_train[:, 2])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [106]:
ypp = lr.predict_proba(np.hstack((X_train, y_train[:, :2])))
print(ypp)
print((ypp >= .5).astype(int))

[[0.35618938 0.64381062]
 [0.5949257  0.4050743 ]
 [0.31517492 0.68482508]
 [0.54990142 0.45009858]
 [0.25894644 0.74105356]]
[[0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]]


In [ ]:
for b in range(2 ** 2):
    y = np.array(list(map(int, np.binary_repr(b, width=2))))

In [8]:
data = load_data('scene')

In [9]:
X, y = data

X = X.toarray()
y = y.toarray()

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=seed)
pcc = ProbabilisticClassifierChain(LogisticRegression())

pcc.fit(X_train, y_train)
y_pred = pcc.predict(X_test)
y_pred_pp = pcc.predict_proba(X_test)
# y_pred_pp = y_pred_pp.reshape(y_pred_pp.shape[0], y_pred_pp.shape[2])
print(ds)
evaluate(y_test, y_pred, y_pred_pp)

NameError: name 'ds' is not defined

In [13]:
evaluate(y_test, y_pred, y_pred_pp)

----------
hamming loss: 
0.10083857442348008
accuracy:
0.6666666666666666
f1 score:
micro
0.7124925283921101
macro
0.728823733607503
precision:
micro
0.7286063569682152
macro
0.7447985584327671
recall:
micro
0.6970760233918128
macro
0.7156021459869816
#--
-> hamming loss:


ValueError: Classification metrics can't handle a mix of multilabel-indicator and unknown targets

In [12]:
y_pred_pp[0]

array([[0.990577  , 0.009423  ],
       [0.98117969, 0.01882031],
       [0.96732898, 0.03267102],
       [0.98906323, 0.01093677],
       [0.21859161, 0.78140839],
       [0.143681  , 0.856319  ]])

In [15]:
y_pred[0]

array([0, 0, 0, 0, 1, 0])

In [17]:
(y_pred_pp[0] >= .5).astype(int)

array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1]])

In [13]:
y_train[0]

array([0, 0, 0, 1, 0, 0], dtype=int64)